# Reference
https://www.heywhale.com/mw/project/5e4bd73f80da780037be6b61

# Dataset Background
    Russian? need translation?
# Data Prepararion
    ERD
    
# Exploratory Data Analysis
    Time

# Time Series Anaylsis

In [4]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


df_item_cat = pd.read_csv("item_categories.csv")
#print(df_item_cat)
df_items = pd.read_csv("items.csv")
#print(df_items)
df_shops = pd.read_csv("shops.csv")
#print(df_shops)
df_sales = pd.read_csv("sales_train.csv")
#print(df_sales.dtypes)

#Mapping of item categories to item table
df_items = pd.merge(df_items,df_item_cat,how="left",on=["item_category_id","item_category_id"])

# Mapping of items and shops to sales transaction table 
df = pd.merge(df_sales, df_items, how="left", on=["item_id","item_id"])
df = pd.merge(df, df_shops, how="left", on=["shop_id","shop_id"])

#remove index columns
df = df.drop(["shop_id","item_id","item_category_id"],axis=1)
#Convert Date field
df['date'] = pd.to_datetime(df['date'])
#compute year, month, day fields
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
#df.to_csv("cleansed_data.csv")
print(df.dtypes)


date                  datetime64[ns]
date_block_num                 int64
item_price                   float64
item_cnt_day                 float64
item_name                     object
item_category_name            object
shop_name                     object
year                           int64
month                          int64
day                            int64
dtype: object


In [2]:
#print(df)
#print(df.dtypes)
df_by_year = df.groupby(["year"],as_index=False).sum(["item_price"])
print(df_by_year)


   year  date_block_num    item_price  item_cnt_day    month       day
0  2013         6999357  9.212613e+08     1562733.0  8357393  20088714
1  2014        18586791  1.043554e+09     1320889.0  7064683  16788104
2  2015        17188912  6.505952e+08      764584.0  3462924   9121572


In [3]:
from pyecharts.charts import Bar
from IPython.core.display import display, HTML
# from pyecharts.globals import CurrentConfig, NotebookType  
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

bar = Bar()
bar.add_xaxis(df_by_year["year"].to_list())
bar.add_yaxis("Item Price", df_by_year["item_price"].to_list())
# render 会生成本地 HTML 文件，默认会在当前目录生成 render.html 文件
# 也可以传入路径参数，如 bar.render("mycharts.html")
# bar.load_javascript();
bar = bar.render()
display(HTML(bar))

In [5]:
df_by_y_m = df.groupby(["year","month"],as_index=False).sum()
df_by_y_m = df_by_y_m.pivot(index="month",columns="year",values="item_price")
df_by_y_m = df_by_y_m.to_dict(orient="list")

In [6]:
print(df_by_y_m)
from pyecharts import options as opts
bar = (Bar()
       .add_xaxis([1,2,3,4,5,6,7,8,9,10,11,12])
       .add_yaxis('2013', df_by_y_m[2013])
       .add_yaxis('2014', df_by_y_m[2014])
       .add_yaxis('2015', df_by_y_m[2015])       
       .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max", name="最大值"),]))
       .set_global_opts(title_opts=opts.TitleOpts(title="Item Price by Year and Month", subtitle="price"))
      )
bar = bar.render()
display(HTML(bar))

{2013: [70601314.09944648, 78276037.9166108, 81096914.31928821, 64065931.74708868, 63551169.617382444, 68657059.77357927, 70280776.22079022, 72280126.1272399, 75192458.83523585, 71511723.66241679, 71996765.37312704, 133750996.8036292], 2014: [74346362.99118166, 79980671.58670628, 83389794.32320061, 71563126.06582402, 76572284.94701071, 75947088.9046997, 72313371.22182254, 83041039.36169231, 81691782.6919338, 75361504.7445927, 100758762.66573344, 168588344.87309378], 2015: [70510913.53070351, 70314002.6679296, 68020084.05501048, 59713087.54806092, 60919855.37541832, 54802044.2963841, 53967769.31255341, 57590892.36052866, 54618374.97439386, 59932860.162888706, 21946834.60039177, 18258444.60563222]}


In [37]:
df_by_item_cat = df.groupby(['item_category_name']).size().reset_index(name='counts').sort_values(by='counts',ascending=False)
print(df_by_item_cat)

dict_by_item_cat = df_by_item_cat.to_dict(orient="list")
print(dict_by_item_cat)

                          item_category_name  counts
40                                Кино - DVD  564652
30             Игры PC - Стандартные издания  351591
55       Музыка - CD локального производства  339585
19                                Игры - PS3  208219
37                            Кино - Blu-Ray  192674
23                           Игры - XBOX 360  146789
28          Игры PC - Дополнительные издания  121539
20                                Игры - PS4   79058
63                  Подарки - Мягкие игрушки   53845
65    Подарки - Настольные игры (компактные)   53227
72                         Подарки - Фигурки   47177
38                         Кино - Blu-Ray 3D   42718
75              Программы - Для дома и офиса   42603
67                        Подарки - Развитие   41706
64                 Подарки - Настольные игры   37635
70            Подарки - Сувениры (в навеску)   35484
41                      Кино - Коллекционное   32748
57                              Музыка - MP3  

In [7]:
bar = (Bar()
       .add_xaxis(dict_by_item_cat['item_category_name'])
       .add_yaxis('Counts', dict_by_item_cat['counts'])      
       .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                        markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max", name="最大值"),]))
       .set_global_opts(title_opts=opts.TitleOpts(title="Item Price by Year and Month", subtitle="price"))
      )
bar = bar.render()
display(HTML(bar))

NameError: name 'dict_by_item_cat' is not defined

In [42]:
from pyecharts.charts import Pie
from pyecharts import options as opts

# Top 10 Item categories
cate = dict_by_item_cat['item_category_name'][:9]
data = dict_by_item_cat['counts'][:9]



pie = (Pie()
       .add('', [list(z) for z in zip(cate, data)],
            radius=["30%", "75%"],
            rosetype="radius")
       .set_global_opts(title_opts=opts.TitleOpts(title="Pie", subtitle="sub-title"))
       .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
      )
pie = pie.render()
display(HTML(pie))